# Llama Document Summary Index With Amazon Bedrock Claude Model
## Tested on an ml.m5.2xlarge

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


This demo showcases the document summary index.

The document summary index will extract a summary from each document and store that summary, as well as all nodes corresponding to the document.

Retrieval can be performed through the LLM or embeddings. We first select the relevant documents to the query based on their summaries. All retrieved nodes corresponding to the selected documents are retrieved.

This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install [LangChain](https://github.com/hwchase17/langchain):

In [2]:
%pip install langchain==0.0.309 --force-reinstall --quiet
%pip install pypdf==3.15.2 --force-reinstall --quiet
%pip install llama-index==0.8.37 --force-reinstall --quiet
%pip install sentence_transformers==2.2.2 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
botocore 1.32.6 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
datasets 2.14.5 requires fsspec[http]<2023.9.0,>=2023.1.0, but you have fsspec 2023.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is inco

In [3]:
%pip install pydantic==1.10.13 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you have urllib3 2.1.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you have urllib3 2.1.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# needed for llamaindex
%pip install openai==0.28 --force-reinstall --quiet 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
botocore 1.32.6 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
datasets 2.14.5 requires fsspec[http]<2023.9.0,>=2023.1.0, but you have fsspec 2023.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you h

In [6]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    get_response_synthesizer,
    set_global_service_context
)
from llama_index.indices.document_summary import DocumentSummaryIndex

### Load Datasets

In [8]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [9]:
import glob
from pypdf import PdfReader, PdfWriter

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()


Now that you have clean PDFs to work with, you will enrich your documents with metadata, then use a process called "chunking" to break up a larger document into small pieces. These small pieces will allow you to generate embeddings without surpassing the input limit of the embedding model.

In this example you will break the document into 1000 character chunks, with a 100 character overlap. This will allow your embeddings to maintain some of its context.

In [10]:
docs = []
for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"data/{filename}"]).load_data()
    doc[0].doc_id = filename.replace(".pdf", "")
    docs.extend(doc)

### Build Document Summary Index

We show two ways of building the index:
- default mode of building the document summary index
- customizing the summary query


In [11]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = 'bedrock-user'

In [12]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [13]:
from llama_index import LangchainEmbedding
from langchain.llms.bedrock import Bedrock # required until llama_index offers direct Bedrock integration
from langchain.embeddings.bedrock import BedrockEmbeddings

model_kwargs_claude = {
    "temperature": 0,
    "top_k": 10,
    "max_tokens_to_sample": 512
}

llm = Bedrock(model_id="anthropic.claude-v2",
              model_kwargs=model_kwargs_claude)

embed_model = LangchainEmbedding(
  BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")
)

service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embed_model, 
                                               chunk_size=512)
set_global_service_context(service_context)

### This will take a few minutes. Please be patient.

In [14]:
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    docs,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
)

current doc id: AMZN-2022-Shareholder-Letter
current doc id: abe6382c-5343-437f-a788-847569559cfd
current doc id: abd8a458-1532-4b70-a5d6-6822ad6e00d6
current doc id: f1dca7cc-ed67-4b94-a7a9-f143bdbeb740
current doc id: 1517b09b-d0b1-4689-a8ae-2faaa3c04dfe
current doc id: ced85ff6-1372-4533-80e2-d39078fb3ee2
current doc id: e26f95f5-a850-4509-8c6b-13f5613421d4
current doc id: AMZN-2021-Shareholder-Letter
current doc id: b8ca6627-7364-4265-b67d-23b85a9e382f
current doc id: 90d7f040-c11b-4020-afb1-e7aaab8d0c7c
current doc id: df6d3df8-003d-42c3-82f5-35580cb1d327
current doc id: 7b17f840-16ea-4f87-84b6-31048fbb834f
current doc id: f8a7b442-678b-440a-adc5-5066a5684f92
current doc id: AMZN-2020-Shareholder-Letter
current doc id: 860875c9-46ff-4ba2-a3a9-d3e490953c3c
current doc id: 7965b5bc-94d8-44b2-bbb0-a33af5fe0a8d
current doc id: df0fd59e-0fdc-4279-8cc2-6c0309bd15af
current doc id: cfff31cf-a33a-45b6-94a7-a88d43d3ba98
current doc id: 5294742e-b635-4865-a335-216586772ffa
current doc id: f

In [15]:
print(len(docs))

25


In [16]:
doc_summary_index.get_document_summary("AMZN-2022-Shareholder-Letter")

" Based on the context, this text is the opening letter from Amazon's 2022 annual shareholder letter written by CEO Andy Jassy. It provides an overview of Amazon's performance and outlook.\n\nSome key points and potential questions this text can answer:\n\n- It discusses Amazon's performance and challenges in 2022, a difficult macroeconomic year. It talks about how Amazon still found ways to grow demand and innovate despite the challenges. Questions it can answer - How did Amazon perform financially in 2022? What were some of the challenges it faced? How did it continue to grow and innovate?\n\n- It provides a historical overview of how Amazon has evolved over 25 years since its early days as just an online book retailer. Questions - How has Amazon's business evolved over time? What new products and services has it introduced over the years? \n\n- It mentions some previous challenging economic times like the 2001 dot-com crash and the 2008-09 recession, and how Amazon balanced short-te

In [17]:
doc_summary_index.storage_context.persist("llama_claude_index")

In [18]:
from llama_index.indices.loading import load_index_from_storage
from llama_index import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="llama_claude_index")
doc_summary_index = load_index_from_storage(storage_context)

### Perform Retrieval from Document Summary Index

We show how to execute queries at a high-level. We also show how to perform retrieval at a lower-level so that you can view the parameters that are in place. We show both LLM-based retrieval and embedding-based retrieval using the document summaries.

In [19]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompt_type import PromptType

CLAUDE_CHOICE_SELECT_PROMPT_TMPL = (
    """
    
    Human: A list of documents is shown below. Each document has a number next to it along  with a summary of the document. A question is also provided. 
    Respond with the respective document number. You should consult to answer the question, in order of relevance, as well as the relevance score. 
    The relevance score is a number from 1-10 based on how relevant you think the document is to the question.\n"
    Do not include any documents that are not relevant to the question. 
    
    Example format: 
    Document 1:\n<summary of document 1>
    Document 2:\n<summary of document 2>
    ...\n\n
    Document 10:\n<summary of document 10>
    
    Question: <question>
    Answer:
    Doc: 9, Relevance: 7
    Doc: 3, Relevance: 4
    Doc: 7, Relevance: 3

    Let's try this now: 
    {context_str}

    Question: {query_str}
    
    Assistant: Answer:"""
)
claude_choice_select_prompt = Prompt(
    CLAUDE_CHOICE_SELECT_PROMPT_TMPL, prompt_type=PromptType.CHOICE_SELECT
)

#### High-level Querying

Note: this uses the default, LLM-based form of retrieval

In [21]:
from utils.llama_custom_parse_choice_select_answer_fn import custom_parse_choice_select_answer_fn
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True, similarity_top_k=10,  verbose=False,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    choice_select_prompt=claude_choice_select_prompt
)


In [22]:
response = query_engine.query("How has AWS evolved?")
print(response)

 Unfortunately there is no information provided in the given context about AWS or how it has evolved over time. The context discusses Amazon's response to the COVID-19 pandemic, including efforts to protect warehouse and delivery workers, adjust operations to meet changing demands, and consider implementing testing. But it does not mention anything specifically about AWS, Amazon's cloud computing platform. Without any relevant information about AWS in the provided sources, I do not have enough context to provide a substantive answer to the question "How has AWS evolved?". I apologize that I cannot provide a useful response without being given background information related to the evolution and history of AWS.


#### LLM-based Retrieval

In [23]:
from llama_index.indices.document_summary import DocumentSummaryIndexRetriever

In [24]:
from utils.llama_custom_parse_choice_select_answer_fn import custom_parse_choice_select_answer_fn
retriever = DocumentSummaryIndexRetriever(
    doc_summary_index,
    choice_select_prompt=claude_choice_select_prompt,
    # choice_batch_size=choice_batch_size,
    # format_node_batch_fn=format_node_batch_fn,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    # service_context=service_context
)

In [25]:
retrieved_nodes = retriever.retrieve("According to the documents, How has AWS evolved?")
print(len(retrieved_nodes))

17


In [26]:
for i, node in enumerate(retrieved_nodes):
    print(node.score)
    print(node.node.get_text())
    print("-----------------------------------------------------------------------------------------------------------------------------------")

10.0
work came from optimizing the connections between this large amount of infrastructure. We also continue
to improve our advanced machine learning algorithms to better predict what customers in various parts of thecountry will need so that we have the right inventory in the right regions at the right time. We’ve recentlycompleted this regional roll out and like the early results. Shorter travel distances mean lower cost to serve,less impact on the environment, and customers getting their orders faster. On the latter, we’re excited aboutseeing more next day and same-day deliveries, and we’re on track to have our fastest Prime delivery speedsever in 2023. Overall, we remain confident about our plans to lower costs, reduce delivery times, and build a
meaningfully larger retail business with healthy operating margins.
AWS has an $85B annualized revenue run rate, is still early in its adoption curve, but at a juncture where it’s
critical to stay focused on what matters most to customers 

In [27]:
%%time
# use retriever as part of a query engine
from llama_index.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode="refine", use_async=True)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("How has AWS evolved?")
print(response)

 The new context from the 2019 Amazon shareholder letter does not contain significant information related to the evolution of AWS over time. The letter focuses on Amazon's response to the COVID-19 pandemic in 2020, specifically efforts to protect employee safety and build testing capacity. While important, this does not provide substantive new information about the historical development of AWS since its launch in 2006.

As such, I do not believe the original answer needs to be refined. It already provides a comprehensive overview of how AWS has evolved, covering:

- Starting as a minimal IaaS offering and expanding into a vast service catalog 

- Leveraging Amazon's infrastructure to bootstrap growth

- Relentless innovation and focus on customer needs

- Maturing from basic IaaS to higher-level PaaS, SaaS, and industry offerings 

- Global expansion for lower latency and localized support

- Continued downward pressure on pricing while increasing performance

In summary, the new cont

#### Embedding-based Retrieval

In [28]:
from llama_index.indices.document_summary import DocumentSummaryIndexEmbeddingRetriever

In [29]:
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    choice_select_prompt=claude_choice_select_prompt,
    # choice_batch_size=choice_batch_size,
    # format_node_batch_fn=format_node_batch_fn,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    #service_context=service_context
)

In [30]:
retrieved_nodes = retriever.retrieve("Human: How has AWS evolved?")

In [31]:
len(retrieved_nodes)

3

In [32]:
for i, node in enumerate(retrieved_nodes):
    
    print(node.node.get_text())
    print("-----------------------------------------------------------------------------------------------------------------------------------")
    

Everybody agreed that having a persistent block store was important to a complete compute service;
however, to have one ready would take an extra year. The question became could we offer customers auseful service where they could get meaningful value before we had all the features we thought they wanted?We decided that the initial launch of EC2 could be feature-poor if we also organized ourselves to listen tocustomers and iterate quickly. This approach works well if you indeed iterate quickly; but, is disastrous if youcan’t. We launched EC2 in 2006 with one instance size, in one data center, in one region of the world, withLinux operating system instances only (no Windows), without monitoring, load balancing, auto-scaling, oryes, persistent storage. EC2 was an initial success, but nowhere near the multi-billion-dollar service it’sbecome until we added the missing capabilities listed above, and then some.
In the early days of AWS, people sometimes asked us why compute wouldn’t just be a